# Standings Test
First we establish a valid oauth token below:

In [5]:
from yahoo_oauth import OAuth2
oauth = OAuth2(None, None, from_file='../oauth2.json')

[2019-08-29 21:50:55,562 DEBUG] [yahoo_oauth.yahoo_oauth.__init__] Checking 
[2019-08-29 21:50:55,568 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 289.9955406188965
[2019-08-29 21:50:55,572 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


This will run more frequently - this will check if the token is valid, if not, refresh. Then, we can try to download info from yahoo.

In [6]:
from bs4 import BeautifulSoup
import xml.etree.ElementTree as et 
import pandas as pd 



if not oauth.token_is_valid():
    oauth.refresh_access_token()
    
df_cols = ["team name", "owner name", "wins", "losses"]
out_df = pd.DataFrame(columns = df_cols)

num_teams = 10

league_ids = [1118202, 1110495, 1112355]

for league_id in league_ids:
    url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nfl.l."+str(league_id)+"/standings"
    # url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nfl.l.1118202/standings"
    # probably migrate to scoreboard when there is data... but unsure
    # url = "https://fantasysports.yahooapis.com/fantasy/v2/league/nfl.l.854917/scoreboard"
    payload = ""
    response = oauth.session.get(url, params=payload)

    # we only want to process data if there's data to pass.
    if not response.status_code == 200:
        print("Status Code Not Valid")
        print(response.text)
    else:

    #   XML Samples which are searchable can be found here: https://developer.yahoo.com/fantasysports/guide/

        soup = BeautifulSoup(response.text, 'lxml')
    #     Remove new line formatting from html to prevent confusion
        clean = str(soup).replace('\n', '')
    #     print(clean)
    #     print(soup.prettify())

    #  Get team Names:
    #     select all name tags beneath team tags
        s_team_names = soup.select('team name')
    #     extract the text from the entire tag
        s_team_names = list(map(lambda x: x.text, s_team_names))
    #     place that in a fancy little pandas series
        s_team_names = pd.Series(s_team_names)

    #  Get owner name:
    #     select all wins tags beneath team tags
        s_nickname = soup.select('team nickname')
    #     extract the text from the entire tag
        s_nickname = list(map(lambda x: x.text, s_nickname))
    #     place that in a fancy little pandas series
        s_nickname = pd.Series(s_nickname)

    #  Get team wins:
    #     select all wins tags beneath team tags
        s_wins = soup.select('team outcome_totals wins')
    #     extract the text from the entire tag
        s_wins = list(map(lambda x: x.text, s_wins))
    #     place that in a fancy little pandas series
        s_wins = pd.Series(s_wins)

    #  Get team losses:
    #     select all losses tags beneath team tags
        s_losses = soup.select('team losses')
    #     extract the text from the entire tag
        s_losses = list(map(lambda x: x.text, s_losses))
    #     place that in a fancy little pandas series
        s_losses = pd.Series(s_losses)

    out_df['team name'] = s_team_names
    out_df['owner name'] = s_nickname
    out_df['wins'] = s_wins
    out_df['losses'] = s_losses
    
    display(out_df)

[2019-08-29 21:51:00,747 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] ELAPSED TIME : 295.17470812797546
[2019-08-29 21:51:00,751 DEBUG] [yahoo_oauth.yahoo_oauth.token_is_valid] TOKEN IS STILL VALID


,team name,owner name,wins,losses
0,Six Sigmas,Alex,0,0
1,Hot Chubb Time Machi,Cameron,0,0
2,The Butt Fumblers,Kurnal,0,0
3,Summer's Team,Summer,0,0
4,Discount Doublecheck,Adith,0,0
5,MikhailMusic,Michael,0,0
6,Shannon Larson,Shannon Larson,0,0
7,I Eat Mop,Ekansh,0,0


,team name,owner name,wins,losses
0,QUESTaceans,Michael,0,0
1,Dustin's Team,Alex,0,0
2,Drew bLees,Dustin,0,0
3,Jessica's Team,Andrew,0,0
4,Mike's Team,Jessica,0,0
5,Haskin' For Trouble,Mike,0,0
6,Srivats's Rad Team,Andrew,0,0
7,Rachel's Team,Srivats,0,0


,team name,owner name,wins,losses
0,Jack's Team,Jack,0,0
1,Bow Tie,Alex,0,0
2,Sherlock Mahomes,Joseph,0,0
3,Ball istic Missiles,Uday,0,0
4,Mayfield of Dreams,Kevin,0,0
5,Bobby's Team,Alexi,0,0
6,Jason's Team,Bobby,0,0
7,Matthew Masison,Jason,0,0
